# Indice

* Mi viaje en programacion: procedural -> orientada a objetos -> funcional
* MapReduce como simplificacion de la computacion distribuida
* Reglas para distribuir el trabajo:
    * Funciones puras (sin mutabilidad)
    * Sin Excepciones (Monads: Option y Try)
* Practica local
    * map
    * flatMap
    * reduce
    * Mutabilidad y condiciones de carrera
    * manejo de excepciones
* En streaming
    * akka-streams
    * dataflow / scio
    * apache spark

# Mi viaje: Basic -> C# -> Scala

## Procedural
GOTO?

## OOP: clases y objetos
perfecto, hasta que tenemos que usar varios procesadores. Lock/mutex nos restringen el uso de multiples procesadores

## Funcional
Incomodo y dificil de razonar en un principio. El mejor camino para sacarle el mejor performance a multiples procesadores

# MapReduce como simplificacion de la computacion distribuida
¿Como procesarias bigdata, si solo pudieras usar personas para procesar todo?

* Enseñar a cada uno como procesar los datos (funcion/algoritmia)
* Fraccionar el trabajo (en chunks) para poder distribuir parte de los datos a cada usuario (shuffle)
* Enviar datos a los usuarios para que los procesen y nos devuelva el resultado (Map o Reduce)
* Juntar todos los datos

## Map: N -> N (chunk -> chunk)
Entrar N registros y sale la misma cantidad

## Reduce: N -> M (chunk -> dato)
Entra N registros y sale M registros. Muchas veces sale 1 solo registro por chunk


# Reglas para distribuir el trabajo

## Funciones puras
Cada dato debe ser calculado independiente de los datos anteriores, para evitar dependencias y condiciones de carrera

## No usar excepciones
Se deben capturar los errores sin que esos arrojen una excepcion al sistema (o uno de los trabajadores podria caerse y dejar el trabajo en un estado incierto).
Los monad (Option/Try) nos facilitan esta tarea (su explicacion en unos minutos)

# Practica local

In [ ]:
// Carguemos dependencias
import $ivy.`org.scala-lang.modules:scala-parallel-collections_2.13:1.0.3`
import scala.collection.parallel.CollectionConverters._

import scala.util.{Try, Success, Failure}

## map

In [ ]:
val duplica: Int => Int = { i => i * 2 }
def triplica(v: Int): Int = v * 3

(1 to 10).map(duplica)
(1 to 10).map(triplica)

In [ ]:
def aumenta(n:Int)(v: Int): Int = n * v

(1 to 10).map(aumenta(2))
(1 to 10).map(aumenta(3))

In [ ]:
val cuatriplica = aumenta(4)(_)

(1 to 10).map(cuatriplica)

## flatMap

In [ ]:
val censo = List (
    "onka,panda,quimera", // casa 1
    "monty", // casa 2
    "canaima", // casa 3
    "obi,max" // casa 4
)

censo.map { perros => perros.split(",") }

In [ ]:
censo.map { perros => perros.split(",") }.flatten

In [ ]:
censo.flatMap { perros => perros.split(",") }

## reduce

In [ ]:
val edades = {    
    val rnd = new java.util.Random
    
    (1 to 10).map { i =>
        val value = rnd.nextInt % 75
        
        if (value > 0)
            value
        else
            value * -1
    }
}

val maximo: (Int, Int) => Int = { case (a, b) => if(a > b) a else b }
val minimo: (Int, Int) => Int = { case (a, b) => if(a < b) a else b }
val suma: (Int, Int) => Int = _ + _

edades.reduce(maximo)
edades.reduce(minimo)
edades.reduce(suma)

In [ ]:
edades.par.reduce(maximo)
edades.par.reduce(minimo)
edades.par.reduce(suma)

## Mutabilidad y condiciones de carrera

In [ ]:
val numTuplas = 8
val testTupla =
    (1 to numTuplas).map { _ => (1, 2) } ++
    (1 to numTuplas).map { _ => (2, 3) }

In [ ]:
// funcion pura
def suma(a: Int, b: Int) = a + b

testTupla.map { case (a,b) => suma(a, b) }

In [ ]:
testTupla.par.map { case (a,b) => suma(a, b) }

In [ ]:
// funcion impura
var acumulado = 0
def acumula(a: Int) = {
    acumulado = acumulado + a
    
    acumulado
}

testTupla.map { case (a,b) => acumula(a) }

In [ ]:
// reseteamos acumulador
acumulado = 0
testTupla.par.map { case (a,b) => acumula(a) }

## Manejo de excepciones

In [ ]:
val intText = List("1", "2", "3", "cuatro", "5", "-1")

In [ ]:
intText.map { t => t.toInt }

In [ ]:
intText.map{ i => try {
        i.toInt
    } catch {
        // que devuelvo si no puedo procesar
        case ex: Throwable => -1
    }
}

In [ ]:
intText.map { i => Try { i.toInt } }

In [ ]:
intText.map { i => Try { i.toInt }.toOption }
intText.flatMap { i => Try { i.toInt }.toOption }

# En streaming

## akka-streams
vamos al notebook akka-streams!

## dataflow / scio y apache spark
ambos siguen los mismos principios de programacion funcional